<a href="https://colab.research.google.com/github/Ted-Shubham/TravelItineraryPlannerAI/blob/main/AIItineraryv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai groq gradio pypdf chromadb sentence-transformers googlemaps pandas


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os

# Set your API keys here
os.environ["GROQ_API_KEY"] = "groq-api-key"
os.environ["GOOGLE_MAPS_API_KEY"] = "google-maps-api-key"


In [3]:
pip install langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully uninstalled langchain-text-splitters-0.3.6
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.20
    Uninstalling langchain-0.3.20:
      Successfully uninstalled langchain-0.3.20


In [4]:
import glob
from langchain.document_loaders import PyPDFLoader

def load_pdfs(pdf_folder):
    """Load and extract text from all PDFs in a folder."""
    pdf_files = glob.glob(f"{pdf_folder}/*.pdf")  # Get all PDF files
    docs = []

    for pdf in pdf_files:
        loader = PyPDFLoader(pdf)  # Load each PDF
        docs.extend(loader.load())  # Extract text

    return docs

# Example Usage: Set the correct folder path
pdf_folder = "/content/TravelItinerary"  # Ensure this matches your folder name
documents = load_pdfs(pdf_folder)

print(f"Loaded {len(documents)} documents from multiple PDFs.")




Loaded 0 documents from multiple PDFs.


In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

# Initialize embedding model
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Store extracted travel info as vectors
vector_db = Chroma.from_documents(documents, embedding_model, persist_directory="travel_db")
vector_db.persist()
retriever = vector_db.as_retriever(search_kwargs={"k": 3})  # Fetch top 3 matches


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
def retrieve_travel_info(query):
    """Retrieve relevant travel details from the vector database."""
    results = retriever.get_relevant_documents(query)
    return "\n".join([doc.page_content for doc in results]) if results else "No relevant info found."

# Test retrieval
print(retrieve_travel_info("Best places to visit in Pathankot"))



visit punjab
56
Fazilka
Pathankot 
paTHaNKOT
Situated at the northern most tip of Gurdaspur, the district of 
Pathankot is a meeting point of three northern states - Punjab, 
Himachal Pradesh and Jammu and Kashmir. Due to its ideal location, 
Pathankot serves as a travel hub for the three northerly states.
visit punjab
56
Fazilka
Pathankot 
paTHaNKOT
Situated at the northern most tip of Gurdaspur, the district of 
Pathankot is a meeting point of three northern states - Punjab, 
Himachal Pradesh and Jammu and Kashmir. Due to its ideal location, 
Pathankot serves as a travel hub for the three northerly states.
visit punjab
56
Fazilka
Pathankot 
paTHaNKOT
Situated at the northern most tip of Gurdaspur, the district of 
Pathankot is a meeting point of three northern states - Punjab, 
Himachal Pradesh and Jammu and Kashmir. Due to its ideal location, 
Pathankot serves as a travel hub for the three northerly states.


In [8]:
print(retriever)


tags=['Chroma', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7e649bdfefd0> search_kwargs={'k': 3}


In [9]:
retriever = vector_db.as_retriever()


In [10]:
print(vector_db._collection.count())  # Check if there are stored vectors


150


In [11]:
vector_db = Chroma.from_documents(documents, embedding_model, persist_directory="travel_db")
vector_db.persist()
retriever = vector_db.as_retriever()


In [8]:
results = vector_db.similarity_search("Best places to visit in Amritsar", k=3)
for res in results:
    print(res.page_content)


visit punjab
14
viSiT puNjab
aMRiTSaR
Inextricably linked with  the history of Sikhism,  Amritsar is one of 
the most revered sites of the world. It was founded as recently as 
the 16th century. Its name is a derivative of the Amrit Sarovar (pool 
of nectar), a man made water tank, in the middle of which stands the  
which stands the Golden T emple, the most sacred Sikh shrine.
Fazilka
visit punjab
14
viSiT puNjab
aMRiTSaR
Inextricably linked with  the history of Sikhism,  Amritsar is one of 
the most revered sites of the world. It was founded as recently as 
the 16th century. Its name is a derivative of the Amrit Sarovar (pool 
of nectar), a man made water tank, in the middle of which stands the  
which stands the Golden T emple, the most sacred Sikh shrine.
Fazilka
visit punjab
14
viSiT puNjab
aMRiTSaR
Inextricably linked with  the history of Sikhism,  Amritsar is one of 
the most revered sites of the world. It was founded as recently as 
the 16th century. Its name is a derivative of

In [9]:
import googlemaps

gmaps = googlemaps.Client(key=os.getenv("GOOGLE_MAPS_API_KEY"))

def get_travel_info(source, destination):
    """Fetch travel time and distance using Distance Matrix API (Free)."""
    result = gmaps.distance_matrix(source, destination, mode="driving")

    if "rows" in result and result["rows"][0]["elements"][0]["status"] == "OK":
        distance = result["rows"][0]["elements"][0]["distance"]["text"]
        duration = result["rows"][0]["elements"][0]["duration"]["text"]
        return f"🚗 Distance: {distance}\n⏳ Estimated Time: {duration}"
    else:
        return "No travel data available."

def find_nearby_places(location, place_type="restaurant"):
    """Find nearby places using Google Places API (Free)."""
    places = gmaps.places_nearby(location=location, radius=5000, type=place_type)

    if "results" in places:
        top_places = [place["name"] for place in places["results"][:5]]
        return f"🔍 Top {place_type}s near {location}: \n" + "\n".join(top_places)
    else:
        return f"No {place_type}s found near {location}."


In [10]:
!pip install -q langchain groq gradio pypdf chromadb


In [11]:
pip install langchain_groq

In [20]:
!pip uninstall -y langchain langchain-core langchain-community langchain-groq
!pip install --upgrade langchain langchain-core langchain-community langchain-groq


Found existing installation: langchain 0.3.21
Uninstalling langchain-0.3.21:
  Successfully uninstalled langchain-0.3.21
Found existing installation: langchain-core 0.3.47
Uninstalling langchain-core-0.3.47:
  Successfully uninstalled langchain-core-0.3.47
Found existing installation: langchain-community 0.3.20
Uninstalling langchain-community-0.3.20:
  Successfully uninstalled langchain-community-0.3.20
Found existing installation: langchain-groq 0.3.1
Uninstalling langchain-groq-0.3.1:
  Successfully uninstalled langchain-groq-0.3.1
  Using cached langchain-0.3.21-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_core-0.3.47-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_community-0.3.20-py3-none-any.whl.metadata (2.4 kB)
  Using cached langchain_groq-0.3.1-py3-none-any.whl.metadata (2.6 kB)
Using cached langchain-0.3.21-py3-none-any.whl (1.0 MB)
Using cached langchain_core-0.3.47-py3-none-any.whl (417 kB)
Using cached langchain_community-0.3.20-py3-none-any.wh

In [23]:
!pip uninstall -y langchain langchain-core langchain-community langchain-groq


Found existing installation: langchain 0.3.21
Uninstalling langchain-0.3.21:
  Successfully uninstalled langchain-0.3.21
Found existing installation: langchain-core 0.3.47
Uninstalling langchain-core-0.3.47:
  Successfully uninstalled langchain-core-0.3.47
Found existing installation: langchain-community 0.3.20
Uninstalling langchain-community-0.3.20:
  Successfully uninstalled langchain-community-0.3.20
Found existing installation: langchain-groq 0.3.1
Uninstalling langchain-groq-0.3.1:
  Successfully uninstalled langchain-groq-0.3.1


In [24]:
!pip install --upgrade langchain langchain-core==0.1.16 langchain-community langchain-groq


  Using cached langchain-0.3.21-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_community-0.3.20-py3-none-any.whl.metadata (2.4 kB)
  Using cached langchain_groq-0.3.1-py3-none-any.whl.metadata (2.6 kB)
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip

In [12]:
from langchain_groq import ChatGroq

print("LangChain Groq is working correctly!")


LangChain Groq is working correctly!


In [ ]:
import os
import gradio as gr
from langchain.schema import SystemMessage, HumanMessage, AIMessage
os.environ["GROQ_API_KEY"] = "groq-api-key"  # Set your Groq API key

# Initialize Groq LLM (default model: Mixtral)
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.7, groq_api_key=os.getenv("GROQ_API_KEY"))

def chat_response(user_input, history):
    messages = [SystemMessage(content="You are an AI travel assistant. Just use the RAG provided information. Don't use your own knowledge.")]
    for human, ai in history:
        messages.append(HumanMessage(content=human))
        messages.append(AIMessage(content=ai))
    messages.append(HumanMessage(content=user_input))

    response = llm.invoke(messages)
    return response.content

# Gradio Interface
gr.ChatInterface(chat_response).launch(debug=True)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e78c2a38d29f7e7963.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import speech_recognition as sr
from gtts import gTTS
import os

recognizer = sr.Recognizer()

def listen():
    """Capture voice input"""
    with sr.Microphone() as source:
        audio = recognizer.listen(source)
    return recognizer.recognize_google(audio)

def speak(text):
    """Convert text to speech"""
    tts = gTTS(text=text, lang='en')
    tts.save("response.mp3")
    os.system("mpg321 response.mp3")


In [ ]:
!pip install langchain openai groq gradio pypdf chromadb sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0